In [1]:
import pandas as pd
import numpy as np

import itertools
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings("ignore")

In [2]:
test_data = pd.read_csv('last_orders_subset.csv')
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5487 entries, 0 to 5486
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order          5487 non-null   int64 
 1   SKU            5487 non-null   int64 
 2   Member         5487 non-null   object
 3   Delivery Date  5487 non-null   object
 4   Name           5487 non-null   object
dtypes: int64(2), object(3)
memory usage: 214.5+ KB


In [3]:
df = pd.read_csv('all_except_last_orders.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28984 entries, 0 to 28983
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Order          28984 non-null  int64 
 1   SKU            28984 non-null  int64 
 2   Member         28984 non-null  object
 3   Delivery Date  28984 non-null  object
 4   Name           28984 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.1+ MB


In [4]:
df['Delivery Date'] = pd.to_datetime(df['Delivery Date'], dayfirst = True)

# Train Test Split

The data is split into last orders (for members who have more than 5 orders) and all but last order.
Previous orders are used to train the recommendation system and the last orders are used for testing.

## Last Order per Member

In [5]:
df.head()

,Order,SKU,Member,Delivery Date,Name
0,8358896,15668375,SSCEHNS,2013-11-02,Root Vegetables
1,8358896,15668467,SSCEHNS,2013-11-02,Beans
2,8358896,15669863,SSCEHNS,2013-11-02,Moong Dal
3,8358896,15669778,SSCEHNS,2013-11-02,Other Dals
4,8358896,15669767,SSCEHNS,2013-11-02,Urad Dal


In [6]:
# identify members who have placed more than 5 orders

member_order_count = df[["Member", "Order"]].drop_duplicates()["Member"].value_counts().reset_index()
members_with_more_than_5_orders = member_order_count[member_order_count["count"] > 5]["Member"]
len(members_with_more_than_5_orders)

166

In [ ]:
# filter the last orders for these members

last_orders = df[df["Member"].isin(members_with_more_than_5_orders)][["Member", "Order", "Delivery Date"]] \
                .drop_duplicates() \
                .sort_values(["Order", "Delivery Date"], ascending = [True, False]) \
                .drop_duplicates(subset = "Member") \
                .reset_index(drop = True)["Order"]

In [8]:
len(last_orders)

166

In [9]:
df_train = df[~df["Order"].isin(last_orders)]
df_train.head()

,Order,SKU,Member,Delivery Date,Name
0,8358896,15668375,SSCEHNS,2013-11-02,Root Vegetables
1,8358896,15668467,SSCEHNS,2013-11-02,Beans
2,8358896,15669863,SSCEHNS,2013-11-02,Moong Dal
3,8358896,15669778,SSCEHNS,2013-11-02,Other Dals
4,8358896,15669767,SSCEHNS,2013-11-02,Urad Dal


In [10]:
df_test = df[df["Order"].isin(last_orders)]
df_test.head()

,Order,SKU,Member,Delivery Date,Name
94,8101324,15668377,SSCEHNS,2014-03-15,Root Vegetables
95,8101324,15668684,SSCEHNS,2014-03-15,Beans
96,8101324,15669778,SSCEHNS,2014-03-15,Other Dals
97,8101324,15668521,SSCEHNS,2014-03-15,Bread
98,8101324,15669865,SSCEHNS,2014-03-15,Other Dals


In [11]:
# df_train.to_csv("df_train.csv", index = False)
# df_test.to_csv("df_test.csv", index = False)

## Simulating Forgotten Items

In [ ]:
# Split each order in the test dataset into basket and forgotten items using train_test_split, selecting 5 items as forgotten and the rest as basket.

basket_list = list()  # List to store basket items for each order
forgotten_list = list()  # List to store forgotten items for each order

for order in df_test["Order"].unique():
    y_dummy = [''] * df_test[df_test["Order"] == order].shape[0]  # Dummy target variable
    basket, forgotten, _, _ = train_test_split(df_test[df_test["Order"] == order], y_dummy, test_size=5, random_state=101)
    basket_list.append(basket)  # Append basket items to the list
    forgotten_list.append(forgotten)  # Append forgotten items to the list

# Combine all basket items and forgotten items into separate dataframes
test_basket_df = pd.concat(basket_list, ignore_index=True)
forgotten_items_df = pd.concat(forgotten_list, ignore_index=True)

In [13]:
test_basket_df["Order"].value_counts()

Order
7362753    26
7392553    20
7566535    19
7391877    16
7428900    16
           ..
7453262     3
7460103     3
7513528     3
7352666     3
8101324     3
Name: count, Length: 166, dtype: int64

In [14]:
forgotten_items_df["Order"].value_counts()

Order
8101324    5
7734044    5
7585543    5
7644998    5
7770723    5
          ..
7370908    5
7391877    5
7360479    5
7370657    5
7737427    5
Name: count, Length: 166, dtype: int64

In [15]:
# test_basket_df.to_csv("test_basket_df.csv", index = False)
# forgotten_items_df.to_csv("forgotten_items_df.csv", index = False)

# Recommendation Engine: User Based Collaborative Filtering

## Member-SKU (User-Item) Matrix

In [16]:
df_train = pd.read_csv("df_train.csv")
df_train.head()

,Order,SKU,Member,Delivery Date,Name
0,8358896,15668375,SSCEHNS,2013-11-02,Root Vegetables
1,8358896,15668467,SSCEHNS,2013-11-02,Beans
2,8358896,15669863,SSCEHNS,2013-11-02,Moong Dal
3,8358896,15669778,SSCEHNS,2013-11-02,Other Dals
4,8358896,15669767,SSCEHNS,2013-11-02,Urad Dal


In [17]:
sku_list = list(df_train["SKU"].unique())
len(sku_list)

632

In [18]:
member_list = list(df_train["Member"].unique())
len(member_list)

638

In [19]:
# Create a DataFrame with all combinations of Member and SKU and their purchase frequency

sku_freq_df = df_train[["Member", "SKU"]].groupby(["Member", "SKU"]) \
                .size() \
                .reset_index(name='frequency') \
                .sort_values(by = ["Member", "frequency"], ascending = [True, False])

sku_freq_df.head()

,Member,SKU,frequency
2,SSCEHNS,7580823,7
8,SSCEHNS,15668377,6
20,SSCEHNS,15669772,6
32,SSCEHNS,15669865,6
7,SSCEHNS,15668375,5


In [20]:
# Create a Member-SKU matrix using a pivot table from sku_freq_df, with 'Member' as rows, 'SKU' as columns, 'frequency' as values, and missing values filled with 0.

ms_matrix = pd.pivot(sku_freq_df, index='Member', columns='SKU', values='frequency').fillna(0)
ms_matrix.head()

SKU,6884195,7541573,7543241,7547271,7547296,7547323,7548497,7548498,7548511,7548730,...,93141092,93141093,93156751,93174226,93176429,93176430,93176431,93289485,93289486,93289487
Member,,,,,,,,,,,,,,,,,,,,,
SSCEHNS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SSCESNS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SSCEWZO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SSCHNCE,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SSCLCSW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Member Similarity Matrix

In [21]:
%%time

# Create a DataFrame representing the cosine similarity matrix for members based on the Member-SKU matrix
member_sim_matrix = pd.DataFrame(cosine_similarity(ms_matrix), index=list(ms_matrix.index), columns=list(ms_matrix.index)) 
member_sim_matrix.head()

CPU times: user 70 ms, sys: 4.52 ms, total: 74.6 ms
Wall time: 9.23 ms


,SSCEHNS,SSCESNS,SSCEWZO,SSCHNCE,SSCLCSW,SSCLLWS,SSCLSCL,SSCLWOR,SSCNHZN,SSCNLEC,...,SWRHZSH,SWRLHOS,SWRLOEC,SWRLSWR,SWRLWHS,SWRNCEH,SWRNCSS,SWRNEHZ,SWRNESE,SWRNHCS
SSCEHNS,1.000000,0.236755,0.102309,0.229930,0.144948,0.139352,0.220770,0.245355,0.035267,0.165320,...,0.144982,0.031103,0.355179,0.187954,0.000000,0.088168,0.095701,0.059860,0.073463,0.063088
SSCESNS,0.236755,1.000000,0.187256,0.117444,0.191014,0.092748,0.213922,0.163299,0.000000,0.132842,...,0.244949,0.048795,0.204656,0.209399,0.060858,0.064550,0.155700,0.039841,0.067229,0.173205
SSCEWZO,0.102309,0.187256,1.000000,0.305446,0.119229,0.043419,0.103855,0.280306,0.282038,0.147411,...,0.178377,0.050762,0.240459,0.168048,0.075974,0.040291,0.170075,0.174078,0.178346,0.288300
SSCHNCE,0.229930,0.117444,0.305446,1.000000,0.112167,0.027232,0.069789,0.095893,0.075810,0.017335,...,0.071919,0.057307,0.190871,0.114180,0.035737,0.075810,0.091430,0.163768,0.049348,0.101710
SSCLCSW,0.144948,0.191014,0.119229,0.112167,1.000000,0.044290,0.166476,0.389906,0.000000,0.338330,...,0.207950,0.062137,0.145638,0.399982,0.000000,0.082199,0.123920,0.076102,0.085612,0.000000


## Test on df_test

In [22]:
test_basket_df = pd.read_csv("test_basket_df.csv")

In [24]:
recc_df_list = list()

for order in list(test_basket_df["Order"].unique()):                                        # Iterate through each order in the test basket DataFrame
    test_order = test_basket_df[test_basket_df["Order"] == order]
    test_member = test_order["Member"].unique()[0]                                          # Get the member associated with the current order
    
    similar_members = member_sim_matrix.loc[test_member].reset_index()                      # Get the similarity scores for the test member
    similar_members.columns = ["member", "similarity"]
    similar_members = similar_members.sort_values("similarity", ascending = False)          # Sort members based on similarity scores
    similar_members_list = list(similar_members.iloc[0:2, 0])                               # Get the top 2 similar members
    
    # Filter the sku_freq_df to include only the SKUs purchased by the similar members
    sku_freq_df_subset = sku_freq_df[sku_freq_df["Member"].isin(similar_members_list)] \
                            .sort_values("frequency", ascending = False)
    
    order_member_sku_df = test_order[["Order", "Member", "SKU"]]

    # Merge the SKU frequency DataFrame with the current order DataFrame to include the current order's SKUs
    sku_freq_with_curr_order = pd.merge(sku_freq_df_subset
                                        , order_member_sku_df
                                        , how = "left"
                                        , on = ["SKU"]
                                    )   

    test_recc = sku_freq_with_curr_order[sku_freq_with_curr_order["Order"].isna()]          # Select SKUs that are not in the current order
    
    test_recc_self = test_recc[test_recc["Member_x"] == test_member]
    test_recc_else = test_recc[test_recc["Member_x"] != test_member]

    test_recc = pd.concat([test_recc_self.iloc[0:5], test_recc_else.iloc[0:5]]              # Select top 5 SKUs for the current member and top 5 for other members
                            , ignore_index = True 
                        )
    test_recc = test_recc.drop_duplicates(subset = "SKU")                                   # Remove duplicates based on SKU
    
    test_recc = test_recc[["SKU"]].iloc[0:5]                                                # Select the top 5 SKUs for recommendation
    test_recc["Order"] = order                                                              # Add the current order to the recommendation DataFrame
    test_recc["Member"] = test_member                                                       # Add the member associated with the current order to the recommendation DataFrame
    recc_df_list.append(test_recc)                                                          # Append the recommendation DataFrame to the list

recc_df = pd.concat(recc_df_list, ignore_index = True)                                      # Final DataFrame containing recommendations for each order

## Evaluation using Recall@5 Metric

In [25]:
forgotten_items_df = pd.read_csv("forgotten_items_df.csv")
forgotten_items_df.head()

,Order,SKU,Member,Delivery Date,Name
0,8101324,15668377,SSCEHNS,2014-03-15,Root Vegetables
1,8101324,15669778,SSCEHNS,2014-03-15,Other Dals
2,8101324,15669814,SSCEHNS,2014-03-15,Jaggery
3,8101324,15669865,SSCEHNS,2014-03-15,Other Dals
4,8101324,15669970,SSCEHNS,2014-03-15,Cashews


In [26]:
# Calculate the recall@5 metric for each order by comparing recommended SKUs with forgotten SKUs and store the results in recall_list

recall_list = list()

for order in list(forgotten_items_df["Order"].unique()):
    forgotten_SKUs = forgotten_items_df[forgotten_items_df["Order"] == order]["SKU"]
    recc_SKUs = recc_df[recc_df["Order"] == order]["SKU"]
    recall = len(set(forgotten_SKUs).intersection(set(recc_SKUs)))/len(set(forgotten_SKUs))
    recall_list.append(recall)

In [27]:
np.mean(recall_list)

0.2469879518072289

## Prepare Kaggle Submission    

In [28]:
# Group the data by Member and SKU, calculate the frequency of each SKU purchased by each Member, and sort by Member and frequency
sku_freq_df = df[["Member", "SKU"]].groupby(["Member", "SKU"]) \
                .size() \
                .reset_index(name='frequency') \
                .sort_values(by = ["Member", "frequency"], ascending = [True, False])

# Create a Member-SKU matrix with Members as rows, SKUs as columns, and frequency as values, filling missing values with 0
ms_matrix = pd.pivot(sku_freq_df, index='Member', columns='SKU', values='frequency').fillna(0)

# Extract the list of unique Members from the dataset
member_list = list(df["Member"].unique())

# Compute the cosine similarity matrix for Members based on the Member-SKU matrix
member_sim_matrix = pd.DataFrame(cosine_similarity(ms_matrix), index=member_list, columns=member_list)

# Initialize an empty list to store recommendation DataFrames for each order
recc_df_list = list()

# Iterate through each unique order in the test dataset
for order in list(test_data["Order"].unique()):
    # Filter the test dataset for the current order
    test_order = test_data[test_data["Order"] == order]
    # Get the Member associated with the current order
    test_member = test_order["Member"].unique()[0]
    
    # Retrieve similarity scores for the test Member and sort them in descending order
    similar_members = member_sim_matrix.loc[test_member].reset_index()
    similar_members.columns = ["member", "similarity"]
    similar_members = similar_members.sort_values("similarity", ascending = False)
    # Select the top 9 similar Members
    similar_members_list = list(similar_members.iloc[0:9, 0])
    
    # Filter the SKU frequency DataFrame to include only SKUs purchased by the similar Members
    sku_freq_df_subset = sku_freq_df[sku_freq_df["Member"] \
                            .isin(similar_members_list)] \
                            .sort_values("frequency", ascending = False)
    
    # Extract the Order, Member, and SKU columns for the current order
    order_member_sku_df = test_order[["Order", "Member", "SKU"]]
    # Merge the SKU frequency DataFrame with the current order DataFrame to include the current order's SKUs
    sku_freq_with_curr_order = pd.merge(sku_freq_df_subset, order_member_sku_df, how = "left", on = ["SKU"])

    # Select SKUs that are not in the current order
    test_recc = sku_freq_with_curr_order[sku_freq_with_curr_order["Order"].isna()] 
    
    # Separate recommendations for the current Member and other Members
    test_recc_self = test_recc[test_recc["Member_x"] == test_member]
    test_recc_else = test_recc[test_recc["Member_x"] != test_member]

    # Combine top 5 SKUs for the current Member and top 5 for other Members, removing duplicates
    test_recc = pd.concat([test_recc_self.iloc[0:5], test_recc_else.iloc[0:5]], ignore_index = True)
    test_recc = test_recc.drop_duplicates(subset = "SKU")
    
    # Select the top 5 SKUs for recommendation and add Order and Member information
    test_recc = test_recc[["SKU"]].iloc[0:5]
    test_recc["Order"] = order
    test_recc["Member"] = test_member
    # Append the recommendation DataFrame to the list
    recc_df_list.append(test_recc)

# Combine all recommendation DataFrames into a single DataFrame
recc_df = pd.concat(recc_df_list, ignore_index = True)

In [29]:
recc_df = recc_df[["Order", "SKU", "Member"]].reset_index().rename(columns = {"index": "ID"})
recc_df["ID"] = recc_df["ID"].apply(lambda x: x+1)
recc_df.head()

,ID,Order,SKU,Member
0,1,7409204,15668465,SWLCNOE
1,2,7409204,15668459,SWLCNOE
2,3,7409204,15668460,SWLCNOE
3,4,7409204,15668467,SWLCNOE
4,5,7409204,15668494,SWLCNOE


In [30]:
# recc_df.to_csv("user_based_collab_filtering_recc.csv", index = False)